In [1]:
%load_ext autoreload
%autoreload 2

In [51]:
import sys
sys.path.append("/home/ray__/ssd/BERT/")
sys.path.append("/home/ray__/CS/org/etherlabs/ai-engine/pkg/")
sys.path.append("../")
from gpt_feat_utils import GPT_Inference

gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/customer_service/epoch3/", device="cpu")
#gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/ether/", device="cpu")

# with open('../topic_testing/cullen_test.json','rb') as f:
#     request = json.load(f)
#     if isinstance(request, str):
#         request = json.loads(request)


ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [52]:
import pickle
import numpy as np
import networkx as nx
# S.E
# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/se/cl_deployment/reduced_entity.pkl","rb"))
# sent_dict = pickle.load(open("/home/ray__/ssd/minds/se/cl_deployment/sent_dict.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/se/cl_deployment/com_map.pkl", "rb"))
# ent_graph = nx.read_gpickle("/home/ray__/ssd/minds/se/se_ent_graph_wcosine_pruned_v3.gpkl")

# Ether
# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/ether/ether_entity_feats_grp.pkl","rb"))
# sent_dict = pickle.load(open("/home/ray__/ssd/minds/ether/ether_sent_dict_v1.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/ether_se/com_map.pkl", "rb"))

# Ether_se

# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/ether_se/se_ether_feats_non_updated_allether.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/ether_se/com_map_h.pkl", "rb"))
# sent_dict = pickle.load(open("/home/ray__/ssd/minds/ether/ether_single_label_dict_v1.pkl","rb"))

# # A.Ient
# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/ai/cl_deployment/reduced_entity.pkl","rb"))
# sent_dict = pickle.load(open("/home/ray__/ssd/minds/ai/cl_deployment/sent_dict.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/ai/cl_deployment/com_map.pkl", "rb"))

#hr
# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/hr/cl_deployment/reduced_entity.pkl","rb"))
# sent_dict = pickle.load(open("/home/ray__/ssd/minds/hr/cl_deployment/sent_dict.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/hr/cl_deployment/com_map.pkl", "rb"))

#marketing
# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/marketing/cl_deployment/reduced_entity.pkl","rb"))
# sent_dict = pickle.load(open("/home/ray__/ssd/minds/marketing/cl_deployment/sent_dict.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/marketing/cl_deployment/com_map.pkl", "rb"))

#product
# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/product/cl_deployment/reduced_entity.pkl","rb"))
# sent_dict = pickle.load(open("/home/ray__/ssd/minds/product/cl_deployment/sent_dict.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/product/cl_deployment/com_map.pkl", "rb"))

#sales
# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/sales/cl_deployment/reduced_entity.pkl","rb"))
# sent_dict = pickle.load(open("/home/ray__/ssd/minds/sales/cl_deployment/sent_dict.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/sales/cl_deployment/com_map.pkl", "rb"))

#customer_service
ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/customer_service/cl_deployment/reduced_entity.pkl","rb"))
sent_dict = pickle.load(open("/home/ray__/ssd/minds/customer_service/cl_deployment/sent_dict.pkl", "rb"))
com_map = pickle.load(open("/home/ray__/ssd/minds/customer_service/cl_deployment/com_map.pkl", "rb"))

# common_entities = ent_fv_full.keys() & com_map.keys()
# ent_fv = {}
# for ent in common_entities:
#     ent_fv[ent] = ent_fv_full[ent]

common_entities = ent_fv_full.keys() & com_map.keys()
ent_fv = {}
for ent in common_entities:
    if ent in sent_dict.keys() and sent_dict[ent]=='PER':
        continue
    ent_fv[ent] = ent_fv_full[ent]

In [53]:
import text_preprocessing.preprocess as tp
from extra_preprocess import preprocess_text
import numpy as np
from scipy.spatial.distance import cosine

def generate_gs(set_list):
    com_freq = {}
    for file in set_list:
        req = json.load(open(file, "r"))
        if isinstance(req, str):
            request = json.loads(req)["body"]
        else:
            request = req["body"]
            
        request["segments"] = sorted(request['segments'], key=lambda kv:kv['startTime'])
        for index, seg in enumerate(request["segments"]):
            request["segments"][index]["originalText"] = " ".join(preprocess_text(seg["originalText"]))
        segments_map = {}
        for index, seg in enumerate(request["segments"]):
            if seg["originalText"] != "":
                segments_map[seg['id']] = seg
                # if len(seg["originalText"].split(". "))==1 and len(seg["originalText"].split(" "))<=6 :
                #continue
                segments_map[seg['id']]["order"] = index
        text = list(map(lambda seg: (seg["originalText"], seg["id"]), [segment for segment in request['segments'] if segment["originalText"]!=""]))
        seg_list = [sent for sent, id  in text]
        segid_list = [id for sent, id in text]
        sent_list = list(map(lambda seg, segid:([sent + ". " for sent in seg.split(". ")],segid), seg_list, segid_list))
        sent_list = [(sent, segid) for seg, segid in sent_list for sent in seg]
        
        segments_fv = {}
        for segments in segments_map.values():
            mod_sent = preprocess_text(segments["originalText"])
            if mod_sent:
                sent_fv = list(map(lambda kv: gpt_model.get_text_feats(kv), mod_sent))
                segments_fv[segments["id"]] = np.mean(sent_fv, axis=0)
        
        
        ent_score = {}
        for segid in segments_fv.keys():
            ent_score[segid] = []
            for ent in ent_fv.keys():
                ent_score[segid].append( (ent, 1 - cosine(segments_fv[segid], ent_fv[ent])))
                
        keys_in_ent_score = ent_score.keys()
        ent_score_sorted = {}
        for segid in keys_in_ent_score:
            sorted_score = sorted(ent_score[segid], key=lambda x: x[1], reverse=True)
            ent_score_sorted[segid] = [i for i,j in sorted_score][:10]
            for ent in [i for i,j in sorted_score][:10]:
                if com_map[ent] in com_freq.keys():
                    com_freq[com_map[ent]] += 1
                else:
                    com_freq[com_map[ent]] = 1
            print ("\n\n\n", segments_map[segid]["originalText"], "\n\n Entities picked: ", [i for i,j in sorted_score][:10], "\n\n community mapped: ",  [com_map[i] for i in [i for i,j in sorted_score][:10]])
        
    return com_freq

In [54]:
import os
dataset = os.listdir("../golden_dataset/customer_service/")

ranked = generate_gs(["../golden_dataset/customer_service/"+i for i in dataset if i!=".ipynb_checkpoints"])




 The story told of this older couple from Maine and every year they go to the Cumberland County Fair this particular year. They were standing by the way their names Martha and Stumpy grinder this particular year there were over by the little airport and Stumpy wanted to take a ride on the airplane and Martha said but Stumpy it is ten dollars and ten dollars is a lot of money and somebody said by Martha. I am going to be XnumberX years old and if I do not do that airplane ride, I is not never gonna go. So they started back and forth and back and forth arguing about this XnumberX actually be XnumberX. Finally the pilot said tell you what I will take you up on that airplane ride for nothing, but I do not want to hear a word from either one of you so Martha said, let us go Stumpy. So they loaded into the plane and the pilot and his mind is saying I am going to have fun with this one. He takes off and he goes straight up in the air straight down flips over. Looks back not a word not a so

 having the conversation all the details about family and when they are traveling but then when they are right there, it is like 

 Entities picked:  ['Mentee', 'Front Desk', 'Rep', 'Whats', 'Reps', 'Portals', 'Names', 'CSM', 'Board', 'Department'] 

 community mapped:  [659, 3502, 3123, 1197, 3123, 1197, 2979, 3502, 1866, 3123]



 Yeah the visual so I would encourage us to explore that how can especially if it is a weak area in our lives to maybe look for ways. How can I work on practicing this we are we easily fall back into habits. Dont we we want to be good at name remembering and we start working at it and then all of a sudden it is like yeah, we want the benefit of it, but we do not necessarily. So it is really important that we stick with it build that skill and it in the payoff not just financially but the payoff is incredible as far as building the relationships and in getting that that Circle it is unbelievable. Okay, so we want to learn their name we talked about smile. We 

 It is about the people it is about serving the people and he tells his little story outstanding in our field and shares the story of how the common man came about. So if you ever have a chance to hear him speak it be well worth it or go on his website just to check out the history, but again another commitment to To that value added and then I had mentioned Stew Leonard is. They also have little books for the kids a little valueadded types of things that so the whole family can be involved. And by the way, they one of the guys that we are talking to down there was telling us about one Thanksgiving or woman had bought her turkey there and she burn it burn it thought the whole thing was just black. 

 Entities picked:  ['Forbes', 'TED', 'CMO', 'English', 'Zappos', 'Harvard Business Review', 'Marketing', 'Gteam', 'Trader Joe', 'Department'] 

 community mapped:  [1230, 2264, 3737, 1125, 463, 2016, 1866, 99, 3420, 3123]



 We had heard it again this morning about partnering do not look a

In [58]:
ranked_sorted = sorted(ranked.items(), key=lambda kv:kv[1], reverse=True)

clusters = {}
temp = []
prev_com = 0
for index,(word,cluster) in enumerate(sorted(com_map.items(), key=lambda kv:kv[1])):
    if prev_com==cluster:
        temp.append(word)
        if index==len(com_map.items())-1:
            clusters[cluster] = temp
    else:
        clusters[prev_com] = temp
        temp = []
        prev_com = cluster
        temp.append(word)
        if index==len(com_map.items())-1:
            clusters[cluster] = temp

In [59]:
for com, freq in ranked_sorted:
    print ("\n\nCommunity: ", com, "  Freq: ", freq ,"\n")
    print (clusters[com])



Community:  68   Freq:  90 

['Online', 'Ecommerce', 'Image', 'CTA', 'CRM', 'Amazon', 'Website', 'Web', 'Internet', 'App', 'Image Source']


Community:  3502   Freq:  90 

['Boardroom', 'Front Desk', 'Ground', 'CSMs', 'CSM']


Community:  1866   Freq:  81 

['UX', 'Metrics', 'HR', 'Marketing', 'Silos', 'Persona', 'Intelligence', 'CSR', 'Board', 'Customer Service Metrics', 'Ideas Forum']


Community:  1290   Freq:  75 

['Advisory Board', 'How-to', 'Blog', 'Management Center', 'Customershowthey', 'Site', 'CTAs', 'Focus']


Community:  1422   Freq:  62 

['Hope', 'Love', 'Canned Response', 'Titles', 'Understand', 'Sorry', 'Yourcustomer', 'Empathy', 'Little', "I'm Sorry", 'Hello', 'Eat', 'Cable', 'Apologies', 'Abc123', 'Apologize', 'Molehills', 'Robotic Jargon', 'Pray']


Community:  1125   Freq:  45 

['Wiki', 'Business', 'EQ', 'English']


Community:  3737   Freq:  42 

['Sales', 'CMO', 'CSO']


Community:  2979   Freq:  42 

['YouGov', 'Emoji', 'Names', 'GIFs', 'Constructive', 'Negat

In [60]:
com_map_selected = {}
for index, (com, freq) in enumerate(ranked_sorted):
    com_map_selected[com] = index

In [61]:
pickle.dump(ranked, open("/home/ray__/ssd/minds/customer_service/cl_deployment/gc.pkl", "wb"))

In [13]:
pickle.dump(com_map_selected, open("/home/ray__/ssd/minds/ether_se/ranked_com_selected_h.pkl", "wb"))

In [62]:
lc = {}
for cls, freq in ranked.items():
    lc[cls] = []

In [48]:
pickle.dump(lc, open("/home/ray__/ssd/minds/sales/cl_deployment/lc.pkl", "wb"))

In [41]:
clusters[5]

['Web',
 'Google',
 'Ios',
 'Janus',
 'Zoom',
 'Gypsy',
 'App',
 'Internet',
 'App Store',
 'Ios App']

In [2]:
import pickle
com_map = pickle.load(open("/home/ray__/ssd/minds/ether/com_map.pkl", "rb"))

In [3]:
ranked_sorted = sorted(com_map.items(), key=lambda kv:kv[1], reverse=True)

clusters = []
temp = []
prev_com = 0
for index,(word,cluster) in enumerate(sorted(com_map.items(), key=lambda kv:kv[1])):
    if prev_com==cluster:
        temp.append(word)
        if index==len(com_map.items())-1:
            clusters.append(temp)
    else:
        clusters.append(temp)
        temp = []
        prev_com = cluster
        temp.append(word)
        if index==len(com_map.items())-1:
            clusters.append(temp)

In [4]:
com_map_refined = {}
for ent,cls in com_map.items():
    if len(clusters[cls])<=2:
        continue
    else:
        com_map_refined[ent] = cls

In [6]:
len(com_map_refined)

160

In [53]:
ent_check = "Harvard Business Review"
ck = []
for ent in ent_fv.keys():
    ck.append((ent, 1-cosine(ent_fv[ent], ent_fv[ent_check])))

In [57]:
for ent, score in sorted(ck, key=lambda kv:kv[1], reverse=True)[:30]:
    print (ent, " -> ", score)

Harvard Business Review  ->  1.0
Mckinsey  ->  0.8903322815895081
Forbes  ->  0.8880045413970947
Cio  ->  0.8862068057060242
Engineering  ->  0.8720834255218506
Digital  ->  0.87077397108078
Software Engineering  ->  0.8672767877578735
C-suite  ->  0.8627213835716248
Man-month  ->  0.8586208820343018
Gartner  ->  0.855764627456665
American  ->  0.8541392087936401
Software  ->  0.8452587723731995
Agiles  ->  0.8377225995063782
Agile Software  ->  0.8330069184303284
10x  ->  0.8307514190673828
Kodak  ->  0.8300771713256836
Goldman Sachs  ->  0.8299168944358826
Tech  ->  0.829247772693634
Finance  ->  0.8286100625991821
Agile  ->  0.8278757929801941
North Star  ->  0.8275061249732971
Org  ->  0.8253639936447144
Accelerate  ->  0.8239707350730896
Itil  ->  0.823801577091217
Lean Startup  ->  0.8228476643562317
Nasa  ->  0.8222607970237732
Italian  ->  0.82192063331604
Thoughtworks  ->  0.821498692035675
Epfl  ->  0.820669412612915
British  ->  0.8203485012054443


# testing sentence to entity similarity

In [43]:
import sys
sys.path.append("../")
from extra_preprocess import preprocess_text

#text = "Everybody wants to create applications using the micro service architecture in this video. But rather than reading out a list of architectural Concepts, we will approach that a completely different place by me telling you a story that you really need to know. Let is go back in time to the old days when we did not have anything called micro services. Theyll go to a time when a computer was big enough to fill the room. Mr. Seen pictures of one of these things you would walk up to that computer. Fast forward a little bit to desktop machines again programs recite it on the desktop machine the programmer application contains all the instructions that are needed to execute. So when people would write applications for it their code would be compiled down in the whole thing would be installed on the computer in one go and it will be installed on the same computer think of purely desktop only apps. Okay, something like Microsoft Word or a text editor that you install on your I mean this has historically influence how we write code when we need to write an application. We would start a new code project and we would add functionality into that project need more functionality add more code. And so the size of the code base for any given application keeps increasing over time. So what starts as a small code base might end up turning into a large complex code base over time and people realize this."
text = "Nothing too fancy, but overtime web applications have started becoming better bigger quicker more complicated bigger scale bigger user base bigger everything and today we have web applications that can you know find something from the whole internet in a matter of milliseconds are fine. What cabs are available around you all over the world in milliseconds. These are incredible Feats if you think about it, and they need incredibly complicated code to be Developed and applied and this complexity becomes harder and harder to maintain. We have nice modular architecture on the code side of things is not that enough to handle the complexity during development time? Well with the type of applications we are talking about the complexity needs to be handled not just at the coding side of things. They also need to be handled at the runtime or execution side of things having a single thing that you would apply. Didnt work anymore this way of having a single application is called the monolithic application or monolithic architecture mono means single lithic means Stone single Stone monolithic. This is the Smosh basically What are some of the disadvantages of this monolithic model first the bigger the deployment the more challenging the deployment Let me Give an example. Let is say you want to push a new feature to your big monolithic application. So among all the code comments that you want to deploy is the single quote comment by this new guy. The company has just hired me not so sure about him. He probably does not know much and he still learning but his first code come it ever is still sitting there. Well, you need to test the whole thing before you would apply the whole application you never know which part of the application that come it might have broken. Well, I am exaggerating here, of course, but the fact remains."
text_p = " ".join(preprocess_text(text))

text_pp = " ".join(tp.preprocess(text, stop_words=True))

In [44]:
text_fv = gpt_model.get_text_feats(text)
ck = []
for ent in ent_fv.keys():
    ck.append((ent, 1-cosine(ent_fv[ent], text_fv)))

In [45]:
[i for i,j in sorted(ck, key=lambda kv:kv[1], reverse=True)[:10]]

['Perfect',
 'Harry',
 'Know',
 'Scotsman',
 'Ugh',
 'Open-closed',
 'Greeks',
 'Wait',
 'Retval',
 'Spaghetti Code']

In [46]:
text_fv = gpt_model.get_text_feats(text_p)
ck = []
for ent in ent_fv.keys():
    ck.append((ent, 1-cosine(ent_fv[ent], text_fv)))

In [47]:
[i for i,j in sorted(ck, key=lambda kv:kv[1], reverse=True)[:10]]

['Perfect',
 'Harry',
 'Know',
 'Scotsman',
 'Ugh',
 'Open-closed',
 'Greeks',
 'Wait',
 'Retval',
 'Spaghetti Code']

In [33]:
text_fv = gpt_model.get_text_feats(text_pp)
ck = []
for ent in ent_fv.keys():
    ck.append((ent, 1-cosine(ent_fv[ent], text_fv)))

In [34]:
[i for i,j in sorted(ck, key=lambda kv:kv[1], reverse=True)[:10]]

['Spaghetti Code',
 'Perfect',
 'Know',
 'God',
 'Ugh',
 'Spaghetti',
 'Netscapes',
 'Universe',
 'Smalltalk',
 'Harry']

In [42]:
sent_dict['Perfect']

['the perfect is the enemy of the good.',
 'the perfect is the enemy of the good.',
 "'perfect is the enemy of good.'",
 'perfect we identified our first object role the entity loader ientityloader interface.',
 "'perfect is the enemy of good.'",
 'finally, swift, originally from apple and used on ios, has begun to see some use in web backends as well, with vapor, kitura and perfect.',
 'since then, a variety of other server frameworks have sprung up such as vapor and perfect, both of which seem to be building polished, feature-rich, and well documented tools for constructing a stack built around swift.',
 "'the perfect is the enemy of the good' — voltaire."]